In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image



In [3]:
!pip install tensorflow-gan

     |████████████████████████████████| 368kB 4.2MB/s 


In [4]:
#@title Inception score set up

'''
From https://github.com/tsc2017/Inception-Score
Code derived from https://github.com/openai/improved-gan/blob/master/inception_score/model.py and https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/gan/python/eval/python/classifier_metrics_impl.py
Usage:
    Call get_inception_score(images, splits=10)
Args:
    images: A numpy array with values ranging from 0 to 255 and shape in the form [N, 3, HEIGHT, WIDTH] where N, HEIGHT and WIDTH can be arbitrary. A dtype of np.uint8 is recommended to save CPU memory.
    splits: The number of splits of the images, default is 10.
Returns:
    Mean and standard deviation of the Inception Score across the splits.
'''

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_gan as tfgan
import os
import functools
import numpy as np
import time
from tensorflow.python.ops import array_ops
# pip install tensorflow-gan
import tensorflow_gan as tfgan
session=tf.compat.v1.InteractiveSession()
# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 64
INCEPTION_TFHUB = 'https://tfhub.dev/tensorflow/tfgan/eval/inception/1'
INCEPTION_OUTPUT = 'logits'

# Run images through Inception.
inception_images = tf.compat.v1.placeholder(tf.float32, [None, 3, None, None], name = 'inception_images')
def inception_logits(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.compat.v1.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    logits = tf.map_fn(
        fn = tfgan.eval.classifier_fn_from_tfhub(INCEPTION_TFHUB, INCEPTION_OUTPUT, True),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 8,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    logits = array_ops.concat(array_ops.unstack(logits), 0)
    return logits

logits=inception_logits()

def get_inception_probs(inps):
    session=tf.get_default_session()
    n_batches = int(np.ceil(float(inps.shape[0]) / BATCH_SIZE))
    preds = np.zeros([inps.shape[0], 1000], dtype = np.float32)
    for i in range(n_batches):
        inp = inps[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] / 255. * 2 - 1
        preds[i * BATCH_SIZE : i * BATCH_SIZE + min(BATCH_SIZE, inp.shape[0])] = session.run(logits,{inception_images: inp})[:, :1000]
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    return preds

def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

def get_inception_score(images, splits=10):
    assert(type(images) == np.ndarray)
    assert(len(images.shape) == 4)
    assert(images.shape[1] == 3)
    assert(np.min(images[0]) >= 0 and np.max(images[0]) > 10), 'Image values should be in the range [0, 255]'
    print('Calculating Inception Score with %i images in %i splits' % (images.shape[0], splits))
    start_time=time.time()
    preds = get_inception_probs(images)
    mean, std = preds2score(preds, splits)
    print('Inception Score calculation time: %f s' % (time.time() - start_time))
    return mean, std  # Reference values: 11.38 for 50000 CIFAR-10 training set images, or mean=11.31, std=0.10 if in 10 splits.


Instructions for updating:
non-resource variables are not supported in the long term

Instructions for updating:
Use keras.layers.Flatten instead.


Instructions for updating:
Use keras.layers.Flatten instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


**DATA preprocessing**

In [5]:
PATH="/content/drive/My Drive/grad_project/approach2/output_4/"
All_FOLDERS=glob.glob(PATH+"*")
print(All_FOLDERS)
print(len(All_FOLDERS))

['/content/drive/My Drive/grad_project/approach2/output_4/0', '/content/drive/My Drive/grad_project/approach2/output_4/1', '/content/drive/My Drive/grad_project/approach2/output_4/2', '/content/drive/My Drive/grad_project/approach2/output_4/3', '/content/drive/My Drive/grad_project/approach2/output_4/4', '/content/drive/My Drive/grad_project/approach2/output_4/5', '/content/drive/My Drive/grad_project/approach2/output_4/6', '/content/drive/My Drive/grad_project/approach2/output_4/7', '/content/drive/My Drive/grad_project/approach2/output_4/8', '/content/drive/My Drive/grad_project/approach2/output_4/9', '/content/drive/My Drive/grad_project/approach2/output_4/10', '/content/drive/My Drive/grad_project/approach2/output_4/11', '/content/drive/My Drive/grad_project/approach2/output_4/12', '/content/drive/My Drive/grad_project/approach2/output_4/13', '/content/drive/My Drive/grad_project/approach2/output_4/14', '/content/drive/My Drive/grad_project/approach2/output_4/15', '/content/drive/M

In [6]:
'''expression_if_true if condition else expression_if_false'''
def custom_order(file_name):
    num=0
    if(file_name[-2:].isdigit()):#r8meen
        num=int(file_name[-2:])
    else:#r8m wa7d
        num=int(file_name[-1])
    return num

#sorted
All_FOLDERS=glob.glob(PATH+"/*")
All_FOLDERS=sorted(All_FOLDERS,key = custom_order )
print(All_FOLDERS)

['/content/drive/My Drive/grad_project/approach2/output_4/0', '/content/drive/My Drive/grad_project/approach2/output_4/1', '/content/drive/My Drive/grad_project/approach2/output_4/2', '/content/drive/My Drive/grad_project/approach2/output_4/3', '/content/drive/My Drive/grad_project/approach2/output_4/4', '/content/drive/My Drive/grad_project/approach2/output_4/5', '/content/drive/My Drive/grad_project/approach2/output_4/6', '/content/drive/My Drive/grad_project/approach2/output_4/7', '/content/drive/My Drive/grad_project/approach2/output_4/8', '/content/drive/My Drive/grad_project/approach2/output_4/9', '/content/drive/My Drive/grad_project/approach2/output_4/10', '/content/drive/My Drive/grad_project/approach2/output_4/11', '/content/drive/My Drive/grad_project/approach2/output_4/12', '/content/drive/My Drive/grad_project/approach2/output_4/13', '/content/drive/My Drive/grad_project/approach2/output_4/14', '/content/drive/My Drive/grad_project/approach2/output_4/15', '/content/drive/M

In [ ]:
#print(len(glob.glob(All_FOLDERS[0]+"/*")))

In [7]:
all_images=[]
for i,folder in enumerate(All_FOLDERS):
    print("image folder #",i)
    folder_images = []
    for img_f in glob.glob(folder+"/*"):
        folder_images.append(np.asarray(Image.open(img_f)))
    folder_images=np.array(folder_images)
    all_images.append(folder_images)



image folder # 0
image folder # 1
image folder # 2
image folder # 3
image folder # 4
image folder # 5
image folder # 6
image folder # 7
image folder # 8
image folder # 9
image folder # 10
image folder # 11
image folder # 12
image folder # 13
image folder # 14
image folder # 15
image folder # 16


In [24]:
print(all_images[15].shape)
print(len(all_images))

(300, 3, 256, 256)
17


In [9]:
for i in range(len(all_images)):
  print(i)
  all_images[i] = np.moveaxis(all_images[i], -1, 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


AxisError: ignored

In [25]:
inception_score=[]
for prediction in all_images[0:15]:
  print(prediction.shape)
  inception_score.append(get_inception_score( (prediction*0.5+0.5) *255))


(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 14.199618 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.332237 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.323343 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.351279 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.339633 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.352066 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.334328 s
(1275, 3, 256, 256)
Calculating Inception Score with 1275 images in 10 splits
Inception Score calculation time: 4.355334 s
(1275, 3, 256, 

In [26]:
inception_score

[(3.0646703, 0.14484411),
 (2.7558901, 0.19100118),
 (2.9365685, 0.1303181),
 (3.03326, 0.17322926),
 (3.046123, 0.13999654),
 (2.9845157, 0.20097847),
 (3.1277394, 0.16289042),
 (3.0306258, 0.11571165),
 (2.9430563, 0.2154127),
 (2.991095, 0.1172187),
 (2.9713178, 0.18324606),
 (3.0079644, 0.15689678),
 (2.9463127, 0.11296062),
 (3.0791788, 0.19804558),
 (2.952847, 0.10405007)]